<a href="https://colab.research.google.com/github/ptkoo/machineLearningJourney/blob/main/Neural_Network_Assignment_Worksheet___Paing_Thet_Ko_65011416.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Simple neural network

In [ ]:
import numpy as np

In [ ]:
class NeuronNetworkLayer:
  # Initializer / Instance Attributes
  def __init__(self, num_input, num_output,activation, initial_gain):
      np.random.seed(42)
      self.inputs = []
      self.outputs = []
      self.weights = np.random.rand(num_output, num_input)*initial_gain
      self.bias = np.random.rand(num_output)* initial_gain
      self.activation = activation


  def sigmoid(self, x):
    # caculate and return output
    output = 1 / (1 + np.exp(-x))
    return output

  def relu(self, x):
    # caculate and return output
    output = np.maximum(0, x)
    return output


  def forward(self, input):
    # calculate forward propagation of layer then store in and return self.outputs
    self.inputs = input
    weighted_sum = np.dot(self.weights, input) + self.bias
    if self.activation == 'sigmoid':
      self.outputs = self.sigmoid(weighted_sum)
    elif self.activation == 'relu':
      self.outputs = self.relu(weighted_sum)
    return self.outputs


  def backprop(self, incoming_gradients, learning_rate):
    # Calculate gradient of Loss with respect to weights, biases and inputs.
    # Use the gradients to update the weights and biases.
    # Return the input_gradients for use in the next layer.
    if self.activation == 'sigmoid':
      derivative = self.outputs * (1 - self.outputs)
    elif self.activation == 'relu':
      derivative = np.where(self.outputs > 0, 1, 0)

    # # the incoming gradients (which are the gradients of the loss with respect to the output of the layer)
    incoming_gradients = np.array([incoming_gradients]) if isinstance(incoming_gradients, np.float64) else incoming_gradients

    grad_weights = np.outer(incoming_gradients * derivative, self.inputs) # Chain Rule

    grad_bias = incoming_gradients * derivative # Chain Rule

    input_gradients = np.dot(incoming_gradients * derivative, self.weights) # Chain Rule

    # Gradient Descent
    self.weights -= learning_rate * grad_weights
    self.bias -= learning_rate * grad_bias.squeeze() # Delete single dimension if (n,1) then (n,)

    return input_gradients

In [ ]:
class NeuralNetwork:
  def __init__(self):
      self.layers = []
      self.output = None

  def add_layer(self, nn_layer):
    # Add layer to neural network
    self.layers.append(nn_layer)

  def forward(self, input):
    # Do forward propagation through entire neural network
    # Calculate and return the final layer_output
    if input.ndim == 1:  # Single sample
      input = input.reshape(1, -1)  # Reshape to 2D array with one row
    # input.reshape(1, -1): If the input is one-dimensional, this line reshapes it into a two-dimensional array with one row and as many columns as needed

    layer_outputs = []
    for sample in input:
      layer_output = sample  # Initialize the layer output for the current sample
      for layer in self.layers:
          layer_output = layer.forward(layer_output)  # Forward pass through the layer
      layer_outputs.append(layer_output)  # Append the output of the current sample

    self.output = np.array(layer_outputs).squeeze()
    return self.output

  # Necessary for back propagation
  def loss(self, output, target):
    # Calculate and return MSE loss
    #loss = np.mean(np.square(output - target))
    output = np.asarray(output)
    target = np.asarray(target)
    # print(output.shape)
    loss = np.mean(np.square(target - output))
    return loss

  def loss_derivative(self, output, target):
    # Calculate and return derivative of MSE loss
    dLoss = (output - target)
    return dLoss

  def backward(self, dLoss, learning_rate):
    # Perform backpropagtion though entire neural network
    # Return the input_gradients in the last layer
    input_gradients = dLoss
    for layer in reversed(self.layers):
      input_gradients = layer.backprop(input_gradients, learning_rate)
    return input_gradients

  def train(self, iterations, train_x, train_y, learning_rate, print_every):
    # Train the neural network
    for i in range(iterations):
      total_loss=0
      for row_x, row_y in zip(train_x, train_y):
        # print('row_x', row_x)
        # print('row_y', row_y)
        # loss = ?
        # total_loss = total_loss + loss
        # Forward pass
        output = self.forward(row_x)
        # Compute loss
        # print(output)
        # print(row_y)
        loss = self.loss(output, row_y)
        total_loss += loss
        # Compute loss derivative
        dLoss = self.loss_derivative(output, row_y)
        # Backward pass
        self.backward(dLoss, learning_rate)

      if i%print_every==0:
        print('total_loss', total_loss)

    return total_loss


## Section 1: Neural Network Algorithm Run Through

In [ ]:
# You have one row of input with two columns. You can think of as column A and column B.
# Here column A=1 and column B=0
input = np.array([1,0])

In [ ]:
layer1 = NeuronNetworkLayer(2,5, 'sigmoid', 0.01)

In [ ]:
layer1_bias = layer1.bias
layer1_bias

array([0.00020584, 0.0096991 , 0.00832443, 0.00212339, 0.00181825])

In [ ]:
assert np.allclose( layer1_bias, np.array([0.00020584, 0.0096991 , 0.00832443, 0.00212339, 0.00181825]) )

In [ ]:
layer1_weights = layer1.weights
layer1_weights

array([[0.0037454 , 0.00950714],
       [0.00731994, 0.00598658],
       [0.00156019, 0.00155995],
       [0.00058084, 0.00866176],
       [0.00601115, 0.00708073]])

In [ ]:
assert np.allclose( layer1_weights, np.array([[0.0037454 , 0.00950714],[0.00731994, 0.00598658],[0.00156019, 0.00155995],[0.00058084, 0.00866176],[0.00601115, 0.00708073]]) )

In [ ]:
layer1_forward = layer1.forward(input)
layer1_forward

array([0.50098781, 0.50425466, 0.50247113, 0.50067606, 0.50195734])

In [ ]:
assert np.allclose( layer1_forward, np.array([0.50098781, 0.50425466, 0.50247113, 0.50067606, 0.50195734]) )

In [ ]:
# let's build a neural network with 3 layers

In [ ]:
layer1 = NeuronNetworkLayer(2,5, 'relu', 0.01)
layer2 = NeuronNetworkLayer(5,10, 'relu', 0.01)
layer3 = NeuronNetworkLayer(10,1, 'relu', 0.01)

In [ ]:
output1 = layer1.forward(input)
output2 = layer2.forward(output1)
output3 = layer3.forward(output2)
output3

array([0.00051439])

In [ ]:
assert np.allclose(output3 , np.array([0.00051439]))

In [ ]:
# Now let's find the error and error gradient

def cal_derivative_and_loss(output3,target):
  print(output3)
  final_output = output3
  loss = (target - final_output)**2
  dLoss = final_output-target
  return loss, dLoss

In [ ]:
loss, dLoss = cal_derivative_and_loss(output3,1)

[0.00051439]


In [ ]:
loss

array([0.99897149])

In [ ]:
dLoss

array([-0.99948561])

In [ ]:
assert np.allclose( loss, 0.9989714888495282)
assert np.allclose( dLoss, -0.9994856121273223)

In [ ]:
# Now let's start backprop on the layers

In [ ]:
input_gradients3 = layer3.backprop(dLoss, 0.1)
input_gradients3

array([-0.00374347, -0.00950225, -0.00731617, -0.00598351, -0.00155938,
       -0.00155914, -0.00058054, -0.00865731, -0.00600806, -0.00707708])

In [ ]:
assert np.allclose( input_gradients3, np.array([-0.00374347, -0.00950225, -0.00731617, -0.00598351, -0.00155938,
       -0.00155914, -0.00058054, -0.00865731, -0.00600806, -0.00707708]))

In [ ]:
input_gradients2 = layer2.backprop(input_gradients3, 0.1)
input_gradients2

array([-0.00019084, -0.00021473, -0.00026229, -0.00029393, -0.00018403])

In [ ]:
assert np.allclose( input_gradients2, np.array([-0.00019084, -0.00021473, -0.00026229, -0.00029393, -0.00018403]))

In [ ]:
input_gradients1 = layer1.backprop(input_gradients2, 0.1)
input_gradients1

array([-3.97277662e-06, -7.35800919e-06])

In [ ]:
assert np.allclose( input_gradients1, np.array([-3.98009953e-06, -7.36642273e-06] ))

## Section 2: Build the Neural Network Class.

In [ ]:
nn_model = NeuralNetwork()
nn_model.add_layer(NeuronNetworkLayer(2,5, 'sigmoid', 0.01) )
nn_model.add_layer(NeuronNetworkLayer(5,10, 'sigmoid', 0.01) )
nn_model.add_layer(NeuronNetworkLayer(10, 1, 'sigmoid', 0.01) )

In [ ]:
input = np.array([1,0])
target = np.array([1])

In [ ]:
forward_result = nn_model.forward(input)
forward_result

array(0.50660746)

In [ ]:
assert np.allclose( forward_result, np.array([0.50660746]) )

In [ ]:
loss = nn_model.loss(forward_result, target)
loss

0.24343619827363822

In [ ]:
loss_derivative = nn_model.loss_derivative(forward_result, target)
loss_derivative

array([-0.49339254])

In [ ]:
assert np.allclose(loss, np.array([0.2434362]) )
assert np.allclose(loss_derivative, np.array([-0.49339254]) )

In [ ]:
backward_result = nn_model.backward(loss_derivative,0.1)
backward_result

array([-3.06340704e-08, -5.67389248e-08])

In [ ]:
assert np.allclose(backward_result, np.array([-3.06340704e-08, -5.67389248e-08]) )

In [ ]:
train_x = np.array( [[1,1]] )
train_y = np.array([[1]] )

total_loss = nn_model.train(10,train_x, train_y, 0.1, 1)

total_loss 0.23278358174415567
total_loss 0.22262202387316252
total_loss 0.21294412619571323
total_loss 0.20373945755988393
total_loss 0.1949951315298813
total_loss 0.18669632207229045
total_loss 0.17882673659198378
total_loss 0.1713690397866621
total_loss 0.16430522529255934
total_loss 0.15761693488389558


In [ ]:
assert total_loss < 0.2

## Section 3: Let's Model Some Logic Gates

In [ ]:
# Let's train an AND Gate

In [ ]:
nn_model = NeuralNetwork()
nn_model.add_layer(NeuronNetworkLayer(2,5, 'sigmoid', 0.01) )
nn_model.add_layer(NeuronNetworkLayer(5,10, 'sigmoid', 0.01) )
nn_model.add_layer(NeuronNetworkLayer(10, 1, 'sigmoid', 0.01) )

In [ ]:
train_x = np.array([ [1,1], [1,0], [0,1], [0,0]])
train_y = np.array([[1], [0], [0], [0] ])

total_loss = nn_model.train(2000,train_x, train_y, 0.5, 500)

total_loss 1.0060940031829388
total_loss 0.7800871908607746
total_loss 0.6578889971126604
total_loss 0.03124644445794698


In [ ]:
assert total_loss < 0.01

In [ ]:
# for AND gate an input of [1,1] should give output of 1
nn_model.forward(np.array([1,1]))

array(0.93399522)

In [ ]:
assert nn_model.forward(np.array([1,1])) > 0.9

In [ ]:
# for AND gate an input of [1,0] should give output of 0
nn_model.forward(np.array([1,0]))

array(0.03395968)

In [ ]:
assert nn_model.forward(np.array([1,0])) < 0.1

In [ ]:
# for AND gate an input of [0,1] should give output of 0
nn_model.forward(np.array([0,1]))

array(0.03426685)

In [ ]:
assert nn_model.forward(np.array([0,1])) < 0.1

In [ ]:
nn_model.forward(np.array([0,0]))

array(0.00019878)

In [ ]:
assert nn_model.forward(np.array([0,0])) < 0.1

In [ ]:
# Let's train an OR Gate and let's make it harder for the model by asking it to predict 0.69

In [ ]:
nn_model = NeuralNetwork()
nn_model.add_layer(NeuronNetworkLayer(2,5, 'relu', 0.01) )
nn_model.add_layer(NeuronNetworkLayer(5,10, 'relu', 0.01) )
nn_model.add_layer(NeuronNetworkLayer(10, 2, 'linear', 0.01) )

In [ ]:
# nn_model.test(np.array([1,0]))

train_x = np.array([ [1,1], [1,0], [0,1], [0,0]])
train_y = np.array([[1,0.69], [1,0.69], [1,0.69], [0,0.69] ])

total_loss = nn_model.train(4000,train_x, train_y, 0.1, 500)

ValueError: operands could not be broadcast together with shapes (2,) (0,) 

In [ ]:
assert total_loss < 0.3

In [ ]:
nn_model.forward(np.array([1,1]))

In [ ]:
nn_model.forward(np.array([1,0]))

In [ ]:
nn_model.forward(np.array([0,1]))

In [ ]:
nn_model.forward(np.array([0,0]))

In [ ]:
assert (nn_model.forward(np.array([1,1]))[0] > 0.7)
assert (nn_model.forward(np.array([1,0]))[0] > 0.7)
assert (nn_model.forward(np.array([0,1]))[0] > 0.7)
assert (nn_model.forward(np.array([0,0]))[0] < 0.3)

assert np.allclose( nn_model.forward(np.array([1,1]))[1], 0.69, rtol=1e-03, atol=1e-03, equal_nan=False)
assert np.allclose( nn_model.forward(np.array([1,0]))[1], 0.69, rtol=1e-03, atol=1e-03, equal_nan=False)
assert np.allclose( nn_model.forward(np.array([0,1]))[1], 0.69, rtol=1e-03, atol=1e-03, equal_nan=False)
assert np.allclose( nn_model.forward(np.array([0,0]))[1], 0.69, rtol=1e-03, atol=1e-03, equal_nan=False)

## Section 4: Linear Regression Prediction with Some Test Lines

In [ ]:
import matplotlib.pyplot as plt

def plotXY(x1,y1,x2,y2):
  # Create the plot
  plt.figure(figsize=(10, 6))  # Optional: Specifies the figure size

  # Plot the first line
  plt.plot(x1, y1, label='Line 1', color='blue', linestyle='-', marker='o')

  # Plot the second line
  plt.plot(x2, y2, label='Line 2', color='red', linestyle='--', marker='x')

  # Adding title
  plt.title('X-Y Plot with Two Lines')

  # Adding X and Y axis labels
  plt.xlabel('X axis label')
  plt.ylabel('Y axis label')

  # Adding a legend
  plt.legend()

  # Show plot
  plt.show()

In [ ]:
# Let's train a straight line.

In [ ]:
# TO DO: Build the neural network

nn_model = NeuralNetwork()
nn_model.add_layer(NeuronNetworkLayer(1,1, 'linear', 0.01) )
# nn_model.add_layer(NeuronNetworkLayer(5,10, 'relu', 0.01) )
# nn_model.add_layer(NeuronNetworkLayer(10, 1, 'relu', 0.01) )

In [ ]:
train_x = np.linspace(-10, 10, 20)
train_y = train_x*2+3

train_x = np.array([[item] for item in train_x])
train_y = np.array([[item] for item in train_y])

# TO DO: Train the model.
#train_x.shape
#total_loss = nn_model.train()
total_loss = nn_model.train(50,train_x, train_y, 0.01, 5)

In [ ]:
assert total_loss < 10

In [ ]:
forward_result = []
for x in train_x:
  forward = nn_model.forward(x)
  forward_result.append(forward)
forward_result

plotXY(train_x, train_y, train_x, forward_result )

In [ ]:
# Let's train a second order polynomial

In [ ]:
# TO DO: Build the neural network

nn_model = NeuralNetwork()
nn_model.add_layer(NeuronNetworkLayer(1, 64, 'relu', 0.0001))
nn_model.add_layer(NeuronNetworkLayer(64, 64, 'sigmoid', 0.1))
nn_model.add_layer(NeuronNetworkLayer(64, 64, 'sigmoid', 0.1))
nn_model.add_layer(NeuronNetworkLayer(64, 1, 'relu', 0.0001))

In [ ]:
train_x = np.linspace(-10, 10, 40)
train_y = train_x*train_x*2+3*train_x+3

train_x = np.array([[item] for item in train_x])
train_y = np.array([[item] for item in train_y])

# TO DO: Train the model.
#total_loss = nn_model.train()
total_loss = nn_model.train(5000,train_x, train_y, 0.00001, 500)

In [ ]:
assert total_loss < 400

In [ ]:
train_x_prediction = []
for x in train_x:
  forward_result = nn_model.forward(x)
  train_x_prediction.append(forward_result)

train_x_prediction
plotXY(train_x, train_y, train_x, train_x_prediction )

## Section 5: Test with Iris Dataset

In [ ]:
# Now let's do the iris datset

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

# Load the Iris dataset
iris = load_iris()

# Convert to DataFrame for easier manipulation
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target
species_mapping = dict(zip(range(3), iris.target_names))
iris_df['species'] = iris_df['species'].map(species_mapping)

In [ ]:
# One-hot encode the 'species' column
species_encoded = pd.get_dummies(iris_df['species'], prefix='species')

iris_df_encoded = pd.concat([iris_df.drop('species', axis=1), species_encoded], axis=1)

train_y = species_encoded.to_numpy()

In [ ]:
iris_df_encoded.head()

In [ ]:
df_x = iris_df_encoded.drop(['species_setosa', 'species_versicolor', 'species_virginica'], axis=1).to_numpy()
df_y = iris_df_encoded[['species_setosa', 'species_versicolor', 'species_virginica']].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.3, random_state=42)

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [ ]:
# TO DO: Create your neural network by completing the code in this cell

nn_model = NeuralNetwork()
nn_model.add_layer(NeuronNetworkLayer(4, 10, 'sigmoid', 0.01))
nn_model.add_layer(NeuronNetworkLayer(10, 20, 'sigmoid', 0.01))
nn_model.add_layer(NeuronNetworkLayer(20, 3, 'linear', 0.01))

In [ ]:
# TO DO: Train the model.
total_loss = nn_model.train(5000,train_x, train_y, 0.01, 500)

In [ ]:
assert total_loss < 5

In [ ]:
def prediction_accuracy(nn_model, train_x, train_y):

  correct=0
  for i in range(len(train_x)):
    prediction = nn_model.forward(train_x[i])
    prediction_index = np.argmax(prediction)
    y_label = np.argmax(train_y[i])
    if y_label == prediction_index:
      correct=correct+1

  return correct/len(train_y)

In [ ]:
train_accuracy = prediction_accuracy(nn_model, train_x, train_y)
train_accuracy

In [ ]:
test_accuracy = prediction_accuracy(nn_model, test_x, test_y)
test_accuracy

In [ ]:
assert train_accuracy > 0.9

In [ ]:
assert test_accuracy > 0.9